##Installation of the libraries

In [1]:
!pip install -q transformers

In [2]:
!pip install biopython

In [3]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

Torch optimization.

##All libraries needed for training

In [4]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2
import torch.nn.functional as F
# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [6]:
file_path = "AF-A0A1D8PD42-F1-model_v4.cif"
file_model = "AF-A0A1D8PD42-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

Desired structure doesn't exists
MSSSNTDNQYPKYINDTTPPTITLKEYDNASWASTTCLDHNPIKNQYIVVVMENPNQIVAIIDQQDNMILDILFKNAHDAHSKQEYSTK


Calculating the angles for the given sequence

In [7]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

Putting angles in a matrix.

In [18]:
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)
psi = np.asarray(psi_angles_list,dtype=np.float32)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [19]:
seq_example =  ' '.join(sequence)
seq_example

'M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K'

##Embedding space creation (using Prot-Bert)

In [20]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

##Single-head self unmasked attention layer

In [37]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [38]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output


In [39]:
class CustomLoss(nn.Module):
    def __init__(self, predicted_angles, angles_tensor):
        super(CustomLoss, self).__init__()
        self.predicted_angles= predicted_angles
        self.angles_tensor= angles_tensor
    def forward(self):
        print(self.predicted_angles)
        print(self.angles_tensor)
        d_list = []
        for i in range(len(self.angles_tensor)):
            x1, y1 = self.predicted_angles[0][i]
            x2, y2 = self.angles_tensor[i]
            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))
            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))
            d = torch.sqrt(ax_x**2 + ax_y**2)
            d_list.append(d)
        d_tensor = torch.stack(d_list)
        return d_tensor.mean()


In [44]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, CustomLoss: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.CustomLoss = CustomLoss
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.SGD(model.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            print(self.angles_tensor)
            loss = CustomLoss()
            loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            #loss_list.append(loss.item())
            print(loss)
            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss



In [45]:
def inc_lr(step_size,init_lr,epoch):
    if(epoch % step_size):
        return init_lr + 0.05

In [46]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 20
dropout_rate = 0.1
model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate)
#model.forward(embeddings)
angles_tensor = torch.from_numpy(angles.T)
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles_tensor)
trainer.train()

tensor([[ 179.9299,   98.1640],
        [ 166.9430,   82.5050],
        [ 175.7644,   79.1991],
        [ 153.2986,   83.0741],
        [ 139.6692,   60.0443],
        [ 176.9491,  102.7947],
        [ 164.3176,   85.2867],
        [ 178.5258,   42.9345],
        [ 171.2007,  127.9657],
        [ 174.5847,  155.6947],
        [ 173.7650,  138.0246],
        [ 172.4886,  137.1495],
        [ 179.3246,  146.5627],
        [ 170.6717,  -34.0441],
        [-176.4155, -179.8297],
        [-173.8806,   -4.4963],
        [-169.4366,  -13.8653],
        [ 173.5445,  110.5009],
        [ 175.3131,  148.9942],
        [ 167.3701,  154.4301],
        [ 176.2569,  130.1698],
        [ 170.4209,  157.8414],
        [ 171.9277,  157.4808],
        [ 175.1485,  -35.6816],
        [ 175.1972,  -41.2917],
        [ 173.9927,  -40.0640],
        [-179.1476,  -50.0214],
        [-179.6617,  -27.9610],
        [-177.6897,   -0.5788],
        [ 176.5145,  132.7397],
        [ 179.8077,  -31.3434],
        

TypeError: CustomLoss.__init__() missing 2 required positional arguments: 'predicted_angles' and 'angles_tensor'

In [ ]:
angles_tensor = torch.from_numpy(angles.T)
predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [ ]:
angles_tensor.size()

torch.Size([89, 2])

In [ ]:
pred = predicted_angles*(180/np.pi)
torch.save(pred,'predictions/GELU_dr_A0A1D8PD42-F1_200_dyn001_AdamW.pt')

In [ ]:
predicted_angles*(180/np.pi)

tensor([[[-5.3158e+00, -8.4865e+01],
         [-2.1541e+01, -1.0659e+02],
         [-1.9706e+01, -1.1475e+02],
         [-2.1093e+01, -1.1402e+02],
         [-2.1229e+01, -1.0726e+02],
         [-5.9279e+00, -7.5901e+01],
         [-1.3012e+01, -9.1103e+01],
         [-2.8945e+00, -3.6999e+01],
         [-2.5739e+00, -3.8034e+01],
         [-2.2872e+00, -3.5981e+01],
         [-3.0082e+00, -3.4816e+01],
         [-5.4122e+00, -3.6587e+01],
         [-3.6470e+00, -3.8944e+01],
         [-2.6871e+00, -3.5675e+01],
         [-4.4649e+00, -3.6202e+01],
         [-3.3329e+00, -3.7021e+01],
         [-3.1330e+00, -3.8703e+01],
         [-3.1629e+00, -7.9383e+01],
         [-2.3305e+00, -3.8988e+01],
         [-2.8487e+00, -3.4812e+01],
         [-4.7120e+00, -3.4776e+01],
         [-2.4408e+00, -3.4859e+01],
         [-3.3734e+00, -4.0475e+01],
         [-3.2046e+00, -3.4718e+01],
         [-2.0182e+00, -3.5554e+01],
         [-1.1058e+00, -3.3644e+01],
         [-1.9041e+00, -3.3950e+01],
 

In [ ]:
angles_tensor*(180/np.pi)

tensor([[ 3.1404,  1.7133],
        [ 2.9137,  1.4400],
        [ 3.0677,  1.3823],
        [ 2.6756,  1.4499],
        [ 2.4377,  1.0480],
        [ 3.0883,  1.7941],
        [ 2.8679,  1.4885],
        [ 3.1159,  0.7493],
        [ 2.9880,  2.2334],
        [ 3.0471,  2.7174],
        [ 3.0328,  2.4090],
        [ 3.0105,  2.3937],
        [ 3.1298,  2.5580],
        [ 2.9788, -0.5942],
        [-3.0790, -3.1386],
        [-3.0348, -0.0785],
        [-2.9572, -0.2420],
        [ 3.0289,  1.9286],
        [ 3.0598,  2.6004],
        [ 2.9212,  2.6953],
        [ 3.0763,  2.2719],
        [ 2.9744,  2.7549],
        [ 3.0007,  2.7486],
        [ 3.0569, -0.6228],
        [ 3.0578, -0.7207],
        [ 3.0367, -0.6992],
        [-3.1267, -0.8730],
        [-3.1357, -0.4880],
        [-3.1013, -0.0101],
        [ 3.0808,  2.3167],
        [ 3.1382, -0.5470],
        [-3.1176, -0.0723],
        [-3.1371, -0.5378],
        [ 3.0780, -0.2015],
        [-3.0377,  0.0956],
        [ 2.9500,  2

In [ ]:
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('AF-A0A1D8PD42-F1-model_v4_pred.pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering

In [ ]:
model = torch.load("/Users/goudarzimandanagmail.com/Desktop/TransformerFromScratch/predictions/GELU_dr_A0A1D8PD42-F1_200_01_AdamW.pt")
model

tensor([[[ 74.1757,  28.8847],
         [ 57.2479,  24.9863],
         [ 86.2554,  34.1374],
         [ 86.4051,  33.7590],
         [ 82.5085,  33.3565],
         [ 71.9535,  29.7960],
         [ 90.0583,  36.6838],
         [ 66.7501,  26.7866],
         [ 80.9579,  32.8670],
         [ 72.4080,  26.1636],
         [ 80.9807,  33.0184],
         [ 78.6896,  28.2421],
         [ 85.5589,  35.1791],
         [ 98.6753,  40.0326],
         [ 86.5369,  34.2731],
         [ 75.8065,  27.3482],
         [ 91.3342,  36.7512],
         [ 88.7715,  36.1417],
         [ 74.8099,  31.3776],
         [ 99.7788,  40.1597],
         [ 74.1245,  32.5588],
         [ 74.0548,  27.1079],
         [ 92.5032,  37.8754],
         [ 86.7356,  34.9321],
         [ 71.7873,  33.4278],
         [ 85.6146,  33.2339],
         [ 85.5991,  35.0503],
         [ 91.0717,  38.0940],
         [ 95.2574,  38.2588],
         [ 73.1474,  35.4199],
         [ 51.2226,  24.2663],
         [ 93.3594,  36.9697],
        

In [ ]:
angles_tensor*(180/np.pi)

tensor([[ 179.9299,   98.1640],
        [ 166.9430,   82.5050],
        [ 175.7644,   79.1991],
        [ 153.2986,   83.0741],
        [ 139.6692,   60.0443],
        [ 176.9491,  102.7947],
        [ 164.3176,   85.2867],
        [ 178.5258,   42.9345],
        [ 171.2007,  127.9657],
        [ 174.5847,  155.6947],
        [ 173.7650,  138.0246],
        [ 172.4886,  137.1495],
        [ 179.3246,  146.5627],
        [ 170.6717,  -34.0441],
        [-176.4155, -179.8297],
        [-173.8806,   -4.4963],
        [-169.4366,  -13.8653],
        [ 173.5445,  110.5009],
        [ 175.3131,  148.9942],
        [ 167.3701,  154.4301],
        [ 176.2569,  130.1698],
        [ 170.4209,  157.8414],
        [ 171.9277,  157.4808],
        [ 175.1485,  -35.6816],
        [ 175.1972,  -41.2917],
        [ 173.9927,  -40.0640],
        [-179.1476,  -50.0214],
        [-179.6617,  -27.9610],
        [-177.6897,   -0.5788],
        [ 176.5145,  132.7397],
        [ 179.8077,  -31.3434],
        